# Dashboard to compare various carbon tracker results

## Datasets
- ukb pilot 

## Pipelines
- FreeSurfer: recon-all (1,2,3 stages)
- FastSurfer: FastSurferCNN (volumetric segmentation)

## Monitoring metrics
- experiment duration in hours
- power consumption in kWh (kilo-watt-hours) 

## Carbon trackers
- EIT
- CC
- CT

In [ ]:
import sys
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from utils.dashboard_utils import * 

from experiment_impact_tracker.data_interface import DataInterface
from experiment_impact_tracker.data_utils import *
from experiment_impact_tracker.data_utils import (load_data_into_frame,
                                                  load_initial_info,
                                                  zip_data_and_info)

In [ ]:
project_dir = '../../'

subject_list_file = f'{project_dir}subject_lists/ukb_pilot_subjects.csv'

tracker_logs_dir = f'{project_dir}tracker_output/'

pipelines = ['FastSurfer/CNN','FastSurfer/recon','FreeSurfer'] # pick from: 'FastSurfer/CNN', 'FastSurfer/recon', 'FreeSurfer'
exp_runs = ['RUN_1']


## Populate paths for different experimental runs
- includes pipeline and run ids

In [ ]:
experiment_sets = []

for pipe in pipelines:
    for run in exp_runs:
        exp_id = f'{pipe}-{run}'
        exp_log_dir = f'{tracker_logs_dir}{pipe}/local_tests/{run}'
        experiment_sets.append((pipe, run, exp_log_dir))

experiment_sets


## Select subjects and trackers

In [ ]:
subject_list = ['sub-000']
trackers = ['EIT','CC'] # pick from 'EIT', 'CC'
read_flops = False
EIT_tracker_summary_df_concat = pd.DataFrame()
CC_tracker_summary_df_concat = pd.DataFrame()

for pipe, run, exp_log_dir in experiment_sets:
    print(f'Reading logs for: {pipe}')

    use_cuda = False
    if pipe == 'FastSurfer/CNN':
        use_cuda = True

    for tracker in trackers: 
        print(f'Using carbon tracker: {tracker}')
        
        subject_log_dirs = []
        for sub in subject_list:
            subject_log_dir = f'{exp_log_dir}/{sub}/{tracker}'
            subject_log_dirs.append(subject_log_dir)

        if tracker == 'EIT':
            EIT_tracker_summary_df, flops_df, power_df = collate_EIT_tracker_data(subject_log_dirs, use_cuda, read_flops)
            EIT_tracker_summary_df['subject_id'] = subject_list
            EIT_tracker_summary_df['pipeline'] = pipe
            EIT_tracker_summary_df['run'] = run
            EIT_tracker_summary_df['tracker'] = tracker
            col_list = list(EIT_tracker_summary_df.columns)
            EIT_tracker_summary_df = EIT_tracker_summary_df[col_list[-4:] + col_list[:-4]]
            EIT_tracker_summary_df_concat = EIT_tracker_summary_df_concat.append(EIT_tracker_summary_df)

        if tracker == 'CC':
            CC_tracker_summary_df = collate_CC_tracker_data(subject_log_dirs)
            CC_tracker_summary_df['subject_id'] = subject_list
            CC_tracker_summary_df['pipeline'] = pipe
            CC_tracker_summary_df['run'] = run
            CC_tracker_summary_df['tracker'] = tracker
            col_list = list(CC_tracker_summary_df.columns)
            CC_tracker_summary_df = CC_tracker_summary_df[col_list[-4:] + col_list[:-4]]
            CC_tracker_summary_df_concat = CC_tracker_summary_df_concat.append(CC_tracker_summary_df)

In [ ]:
EIT_tracker_summary_df_concat

In [ ]:
CC_tracker_summary_df_concat